In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

In [2]:
# Replace the path with the correct path for your data.
y2015 = pd.read_csv('~/Desktop/Data_Science/LoanStats3d.csv',
    skipinitialspace=True,
    header=1
)

# Note the warning about dtypes.

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
y2015.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,20000.0,20000.0,20000.0,36 months,9.80%,643.47,B,B3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,15000.0,15000.0,15000.0,60 months,13.44%,344.69,C,C3,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,7200.0,7200.0,7200.0,36 months,11.48%,237.36,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,15000.0,15000.0,15000.0,36 months,8.49%,473.45,B,B1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,16000.0,16000.0,16000.0,60 months,8.49%,328.19,B,B1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Data Cleaning
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

id
2
term
2
int_rate
111
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
2
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
41
next_pymnt_d
5
last_credit_pull_d
42
application_type
2
verification_status_joint
1
hardship_flag
2
hardship_type
1
hardship_reason
9
hardship_status
3
hardship_start_date
16
hardship_end_date
17
payment_plan_start_date
17
hardship_loan_status
4
disbursement_method
1
debt_settlement_flag
2
debt_settlement_flag_date
34
settlement_status
3
settlement_date
37


In [5]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [6]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
421092,NaN,NaN,12000.0,12000.0,12000.0,60 months,19.99,317.86,E,1 year,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
421093,NaN,NaN,13000.0,13000.0,13000.0,60 months,15.99,316.07,D,5 years,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
421094,NaN,NaN,20000.0,20000.0,20000.0,36 months,11.99,664.20,B,10+ years,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
421095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [8]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,settlement_date_May-2018,settlement_date_Nov-2015,settlement_date_Nov-2016,settlement_date_Nov-2017,settlement_date_Oct-2015,settlement_date_Oct-2016,settlement_date_Oct-2017,settlement_date_Sep-2015,settlement_date_Sep-2016,settlement_date_Sep-2017
0,NaN,NaN,20000.0,20000.0,20000.0,9.80,643.47,75000.0,23.45,0.0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,15000.0,15000.0,15000.0,13.44,344.69,77213.0,18.17,0.0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,7200.0,7200.0,7200.0,11.48,237.36,72500.0,8.47,0.0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,15000.0,15000.0,15000.0,8.49,473.45,110000.0,13.24,0.0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,NaN,16000.0,16000.0,16000.0,8.49,328.19,62000.0,28.92,0.0,...,0,0,0,0,0,0,0,0,0,0
5,NaN,NaN,6000.0,6000.0,6000.0,7.49,186.61,100000.0,13.28,1.0,...,0,0,0,0,0,0,0,0,0,0
6,NaN,NaN,20000.0,20000.0,20000.0,15.77,700.88,70000.0,22.21,0.0,...,0,0,0,0,0,0,0,0,0,0
7,NaN,NaN,15000.0,15000.0,15000.0,5.32,451.73,80000.0,20.18,0.0,...,0,0,0,0,0,0,0,0,0,0
8,NaN,NaN,14000.0,14000.0,14000.0,9.80,450.43,100000.0,15.97,0.0,...,0,0,0,0,0,0,0,0,0,0
9,NaN,NaN,3600.0,3600.0,3600.0,11.48,118.68,85000.0,17.44,0.0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#Random Forest Model

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.99097644, 0.98893427, 0.98957492, 0.98952743, 0.98990739,
       0.9892897 , 0.98218908, 0.98667712, 0.98024128, 0.98639181])

## Assignment

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

### test correlation

In [10]:
#test correlation of variables
correlation_matrix = y2015.corr()
display(correlation_matrix)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,settlement_amount,settlement_percentage,settlement_term
id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
member_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loan_amnt,NaN,NaN,1.000000,1.000000,0.999994,0.140476,0.941174,0.305734,0.018281,-0.010424,...,NaN,0.728733,NaN,0.050245,0.721406,0.839429,0.521167,0.830945,-0.039064,0.263459
funded_amnt,NaN,NaN,1.000000,1.000000,0.999994,0.140476,0.941174,0.305734,0.018281,-0.010424,...,NaN,0.728733,NaN,0.050245,0.721406,0.839429,0.521167,0.830945,-0.039064,0.263459
funded_amnt_inv,NaN,NaN,0.999994,0.999994,1.000000,0.140114,0.941156,0.305803,0.018120,-0.010448,...,NaN,0.728339,NaN,0.050239,0.721020,0.839218,0.521106,0.830811,-0.038946,0.263361
int_rate,NaN,NaN,0.140476,0.140476,0.140114,1.000000,0.124393,-0.090355,0.203331,0.043713,...,NaN,0.631139,NaN,0.125322,0.625184,0.375934,0.347945,0.372264,-0.002757,0.121287
installment,NaN,NaN,0.941174,0.941174,0.941156,0.124393,1.000000,0.297394,0.005532,-0.002128,...,NaN,0.574945,NaN,0.065132,0.561304,0.638579,0.387524,0.719708,-0.030039,0.210364
annual_inc,NaN,NaN,0.305734,0.305734,0.305803,-0.090355,0.297394,1.000000,-0.174934,0.037910,...,NaN,0.274781,NaN,0.004239,0.264244,0.392691,0.213710,0.140311,-0.003252,0.024309
dti,NaN,NaN,0.018281,0.018281,0.018120,0.203331,0.005532,-0.174934,1.000000,-0.019319,...,NaN,0.094497,NaN,0.008493,0.095277,0.052364,0.081008,0.062642,0.018496,0.020050
delinq_2yrs,NaN,NaN,-0.010424,-0.010424,-0.010448,0.043713,-0.002128,0.037910,-0.019319,1.000000,...,NaN,-0.010366,NaN,0.028451,-0.012243,-0.015110,-0.013823,-0.007074,-0.002527,-0.000587


In [26]:
#test each column's correlation with target, print column correlation above 0.75
from pandas.api.types import is_numeric_dtype

for column in X:
    if is_numeric_dtype(X[column]):
        correlation = Y.corr(X[column])
        if correlation > 0.75:
            print(column, correlation)

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [14]:
X.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate',
       'installment', 'annual_inc', 'delinq_2yrs', 'inq_last_6mths',
       'open_acc', 'pub_rec',
       ...
       'settlement_date_May-2018', 'settlement_date_Nov-2015',
       'settlement_date_Nov-2016', 'settlement_date_Nov-2017',
       'settlement_date_Oct-2015', 'settlement_date_Oct-2016',
       'settlement_date_Oct-2017', 'settlement_date_Sep-2015',
       'settlement_date_Sep-2016', 'settlement_date_Sep-2017'],
      dtype='object', length=376)

In [19]:
data = y2015[['loan_status', 'annual_inc']]
correlation = data.corr(method='pearson')
print(correlation)
single_correlation = correlation[0][1]
print(single_correlation)

            annual_inc
annual_inc         1.0


KeyError: 0

In [17]:
from sklearn.feature_selection import RFE

# Pass any estimator to the RFE constructor
selector = RFE(rfc)

In [23]:
#Try first ten features
ten1 = X.iloc[:, 0:10]

selector = selector.fit(ten1, Y)

In [24]:
rankings = pd.DataFrame({'Features': ten1.columns, 'Ranking' : selector.ranking_})
rankings.sort_values('Ranking')

,Features,Ranking
2,funded_amnt_inv,1
3,int_rate,1
4,installment,1
5,annual_inc,1
8,open_acc,1
0,loan_amnt,2
1,funded_amnt,3
6,delinq_2yrs,4
7,inq_last_6mths,5
9,pub_rec,6


In [25]:
#Test out #1 ranked features from first ten options

#Random Forest Model

rfc = ensemble.RandomForestClassifier()
rfc1_X = X[['funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'open_acc']]
Y = y2015['loan_status']

score = cross_val_score(rfc, refc1_X, Y, cv=10)
print("Cross Validation Scores:\n", score)
print("\nCross Validation Average: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Cross Validation Scores:
 [0.41567724 0.43685885 0.43797198 0.43714082 0.44599858 0.46545869
 0.484552   0.48221241 0.52443716 0.55826822]

Cross Validation Average: 0.47 (+/- 0.08)


In [28]:
#Try features 10-20

ten2 = X.iloc[:, 10:20]

selector = selector.fit(ten2, Y)

In [15]:
rankings = pd.DataFrame({'Features': ten2.columns, 'Ranking' : selector.ranking_})
rankings.sort_values('Ranking')

NameError: name 'ten2' is not defined

In [19]:
#Test out #1 ranked features from first ten options

#Random Forest Model

rfc = ensemble.RandomForestClassifier()
X = y2015[['funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'open_acc', 'out_prncp', 'out_prncp_inv', 'total_rec_prncp', 'total_rec_int', 'recoveries']]
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.94652356, 0.98534859, 0.98351935, 0.98366184, 0.98245072,
       0.98256905, 0.98197535, 0.98149995, 0.97698775, 0.9807158 ])

In [21]:
correlation_matrix = X.corr()
display(correlation_matrix)

,funded_amnt_inv,int_rate,installment,annual_inc,open_acc,out_prncp,out_prncp_inv,total_rec_prncp,total_rec_int,recoveries
funded_amnt_inv,1.000000,0.140114,0.941156,0.305803,0.188212,0.370322,0.370345,0.741222,0.692954,0.135308
int_rate,0.140114,1.000000,0.124393,-0.090355,-0.012967,0.153864,0.153750,-0.104668,0.467611,0.173040
installment,0.941156,0.124393,1.000000,0.297394,0.169994,0.204616,0.204609,0.774728,0.585503,0.131749
annual_inc,0.305803,-0.090355,0.297394,1.000000,0.122328,0.084156,0.084182,0.271068,0.146355,0.018652
open_acc,0.188212,-0.012967,0.169994,0.122328,1.000000,0.066759,0.066764,0.129737,0.115236,0.037349
out_prncp,0.370322,0.153864,0.204616,0.084156,0.066759,1.000000,0.999998,-0.014559,0.652024,-0.100046
out_prncp_inv,0.370345,0.153750,0.204609,0.084182,0.066764,0.999998,1.000000,-0.014533,0.651977,-0.100049
total_rec_prncp,0.741222,-0.104668,0.774728,0.271068,0.129737,-0.014559,-0.014533,1.000000,0.368087,-0.199240
total_rec_int,0.692954,0.467611,0.585503,0.146355,0.115236,0.652024,0.651977,0.368087,1.000000,0.017676
recoveries,0.135308,0.173040,0.131749,0.018652,0.037349,-0.100046,-0.100049,-0.199240,0.017676,1.000000


In [22]:
#Remove 'funded_amnt_inv' as it has strong correlation to 'installment'
rfc = ensemble.RandomForestClassifier()
X = y2015[['int_rate', 'installment', 'annual_inc', 'open_acc', 'out_prncp', 'out_prncp_inv', 'total_rec_prncp', 'total_rec_int', 'recoveries']]
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.95101159, 0.98513488, 0.98420803, 0.98318689, 0.98202327,
       0.9819516 , 0.98169037, 0.98081125, 0.97603781, 0.97986083])

In [23]:
#Remove 'out_prncp_inv' as it has strong correlation to 'out_prncp'
rfc = ensemble.RandomForestClassifier()
X = y2015[['int_rate', 'installment', 'annual_inc', 'open_acc', 'out_prncp', 'total_rec_prncp', 'total_rec_int', 'recoveries']]
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.96295593, 0.98409005, 0.98280693, 0.9831394 , 0.98240323,
       0.98185661, 0.98112043, 0.98047877, 0.97715398, 0.97948085])

In [24]:
#Remove 'total_rec_prncp' as it has strong-ish correlation to 'installment'
rfc = ensemble.RandomForestClassifier()
X = y2015[['int_rate', 'installment', 'annual_inc', 'open_acc', 'out_prncp', 'total_rec_int', 'recoveries']]
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


array([0.89789134, 0.90883834, 0.90921396, 0.90669675, 0.91220613,
       0.91567123, 0.91519628, 0.91450556, 0.91277192, 0.92160448])

In [25]:
#Average Cross Validation Score
score = cross_val_score(rfc, X, Y, cv=10)
print("Average Cross Validation Score: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Average Cross Validation Score: 0.91 (+/- 0.01)


In [28]:
import statsmodels.formula.api as smf 
from statsmodels.sandbox.regression.predstd import wls_prediction_std

test = y2015[['loan_status','int_rate', 'installment', 'annual_inc', 'open_acc', 'out_prncp', 'total_rec_int', 'recoveries']]
test = test.dropna(axis=1)

linear_formula = 'loan_status ~ int_rate+installment+annual_inc+open_acc+out_prncp+total_rec_int+recoveries'

lm = smf.ols(formula=linear_formula, data=test).fit()

print(lm.params)

print(lm.pvalues)

                          0             1             2             3  \
Intercept     -7.873596e-02  4.128316e-01 -1.003415e-05  6.737134e-01   
int_rate       1.832483e-02 -1.069365e-02  4.689401e-06 -9.201165e-03   
installment    2.812045e-06 -1.549845e-04 -8.929793e-09  1.495876e-04   
annual_inc    -6.300366e-08 -1.063328e-07  4.127947e-11  1.755707e-07   
open_acc       2.067807e-03 -1.560231e-03 -1.239904e-06 -5.464910e-04   
out_prncp     -1.357123e-05  7.836158e-05  5.996294e-09 -7.061951e-05   
total_rec_int -8.117093e-06 -8.426858e-06 -7.892661e-09  1.903861e-05   
recoveries     2.038784e-04 -2.447745e-05 -6.002712e-09 -1.764840e-04   

                          4             5             6  
Intercept     -1.053119e-03 -6.169383e-04 -6.128895e-03  
int_rate       3.374971e-04  1.888494e-04  1.038954e-03  
installment   -1.057703e-06 -4.636520e-07  4.115111e-06  
annual_inc    -1.769070e-09 -5.785254e-10 -3.927900e-09  
open_acc       1.125771e-05 -2.776333e-06  3.167424e

ValueError: shapes (421095,7) and (421095,7) not aligned: 7 (dim 1) != 421095 (dim 0)

In [ ]:
#Try recursive feature selection on all data (first try didn't execute)
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)
selector = RFE(rfc)
selector = selector.fit(X, Y)

In [ ]:
Y.shape

In [ ]:
sample_size = int(round(len(X.columns)/10))
sample_start = 0
sample_end = sample_size
while sample_end <= len(X.columns):
    sample = X.iloc[:,sample_start:sample_end]
    X = pd.get_dummies(sample)
    X = X.dropna(axis=1)
    print(cross_val_score(rfc, X, Y, cv=10))
    sample_start = sample_end
    sample_end += sample_size

### PCA attempt

In [11]:
import math
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

In [ ]:
# Make the scatterplot matrix.

# Setting the default plot aesthetics to be prettier.
sns.set_style("white")

# Take a subset of the data for PCA.
df_pca = y2015.dropna()

t = sns.regplot(
    'ppltrst',
    'pplfair',
    df_pca,
    df_pca,
    x_jitter=.49,
    y_jitter=.49,
    fit_reg=False
)
t.set(xlim=(-1, 11), ylim=(-1, 11))
t.axhline(0, color='k', linestyle='-', linewidth=2)
t.axvline(0, color='k', linestyle='-', linewidth=2)
t.axes.set_title('Raw data')
plt.show()

# Standardizing variables by subtracting the mean and dividing by the standard
# deviation. Now both variables are on the same scale.
df_pca['ppltrst_z'] = (df_pca['ppltrst'] - df_pca['ppltrst'].mean()) / df_pca['ppltrst'].std()
df_pca['pplfair_z'] = (df_pca['pplfair'] - df_pca['pplfair'].mean()) / df_pca['pplfair'].std()

t = sns.regplot(
    'ppltrst_z',
    'pplfair_z',
    df_pca,
    x_jitter=.49,
    y_jitter=.49,
    fit_reg=False
)
t.axhline(0, color='k', linestyle='-', linewidth=2)
t.axvline(0, color='k', linestyle='-', linewidth=2)
t.axes.set_title('Standardized data')
plt.show()



In [12]:
# drop missing values 
df_pca = y2015.dropna()

# Normalize the data so that all variables have a mean of 0 and standard deviation of 1.
X = StandardScaler().fit_transform(df_pca)

# The NumPy covariance function assumes that variables are represented by rows, not columns, so we transpose X.
Xt = X.T
Cx = np.cov(Xt)
print('Covariance Matrix:\n', Cx)

ValueError: Found array with 0 sample(s) (shape=(0, 137)) while a minimum of 1 is required by StandardScaler.